In [1]:
!csvstat dim_acct.csv

  1. ACCT_ID
	<class 'int'>
	Nulls: False
	Min: 112110616977376
	Max: 4611679791824819900
	Sum: 22730670545637110906347
	Mean: 2.2953317727594778e+18
	Median: 2316664370354537529
	Standard Deviation: 1.333126148041693e+18
	Unique values: 9903
  2. COUNTRY_CODE
	<class 'str'>
	Nulls: False
	Unique values: 11
	5 most frequent values:
		US:	8010
		CA:	1254
		BR:	335
		MX:	131
		AR:	87
	Max length: 2
  3. LANGUAGE_CODE
	<class 'str'>
	Nulls: False
	Values: fr, pt, en, es
  4. OPT_IN_FLAG
	<class 'int'>
	Nulls: False
	Values: 0, 1

Row count: 9903


In [2]:
!csvstat dim_product.csv

  1. sku
	<class 'int'>
	Nulls: False
	Min: 24996
	Max: 76813
	Sum: 47339896
	Mean: 63886.49932523617
	Median: 63589
	Standard Deviation: 8827.863736954238
	Unique values: 709
	5 most frequent values:
		73404:	3
		55210:	2
		73692:	2
		74238:	2
		69489:	2
  2. game_name
	<class 'str'>
	Nulls: False
	Values: Fifa 16, Destiny, Fifa 15

Row count: 741


In [3]:
!csvstat fct_transaction.csv

  1. ACCT_ID
	<class 'int'>
	Nulls: False
	Min: 112110616977376
	Max: 4611679791824819900
	Sum: 22832397974228024195145
	Mean: 2.2958670662873833e+18
	Median: 2318601711805822924
	Standard Deviation: 1.3333984314874568e+18
	Unique values: 9903
	5 most frequent values:
		4325452977810999850:	2
		3622466727119658779:	2
		237755680392739618:	2
		4024747712087038459:	2
		1102743685708855971:	2
  2. TRANSACTION_TIMESTAMP
	<class 'datetime.datetime'>
	Nulls: False
	Min: 2015-10-18 00:00:00
	Max: 2015-11-16 16:24:00
	Unique values: 8469
	5 most frequent values:
		2015-10-18 00:00:00:	6
		2015-10-18 00:50:00:	4
		2015-10-25 00:40:00:	4
		2015-11-06 01:53:00:	4
		2015-11-01 23:00:00:	4
  3. TRANSACTION_VALUE
	<class 'float'>
	Nulls: False
	Min: 0.36567
	Max: 149.99
	Sum: 189099.6233399983
	Mean: 19.01454231674191
	Median: 9.99
	Standard Deviation: 22.599633708954638
	Unique values: 385
	5 most frequent values:
		9.99:	1122
		1.99:	990
		19.99:	951
		59.99:	742
		4.99:	731
  4. SKU
	<class 'int'

In [4]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [5]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [6]:
!createdb -U dbuser CRM

createdb: database creation failed: ERROR:  database "CRM" already exists


In [7]:
%sql postgresql://dbuser@localhost:5432/CRM

'Connected: dbuser@CRM'

In [32]:
%%sql
DROP TABLE IF EXISTS account;
CREATE TABLE account (
    ACCT_ID FLOAT,
    COUNTRY_CODE VARCHAR(2),
    LANGUAGE_CODE VARCHAR(2),
    OPT_IN_FLAG INT,
    PRIMARY KEY (ACCT_ID)
    )

Done.
Done.


[]

In [33]:
!pwd

/home/jovyan/work


In [34]:
%%sql
COPY account FROM '/home/jovyan/work/dim_acct.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

9903 rows affected.


[]

In [35]:
%%sql
SELECT count(account.ACCT_ID) FROM account;

1 rows affected.


count
9903


In [38]:
%%sql
SELECT COUNT(DISTINCT(ACCT_ID)) from account

1 rows affected.


count
9903


In [42]:
%%sql
DROP TABLE IF EXISTS product;
CREATE TABLE product (
    SKU INT,
    GAME_NAME VARCHAR(10)
    )

Done.
Done.


[]

In [43]:
%%sql
COPY product FROM '/home/jovyan/work/dim_product.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

741 rows affected.


[]

In [15]:
%%sql
SELECT count(product.SKU) FROM product;

1 rows affected.


count
741


In [44]:
%%sql
SELECT COUNT(DISTINCT(SKU)) from product

1 rows affected.


count
709


In [45]:
%%sql
ALTER TABLE product
ADD COLUMN product_id SERIAL PRIMARY KEY;

Done.


[]

In [47]:
%%sql
DROP TABLE IF EXISTS transaction_fct;
CREATE TABLE transaction_fct (
    ACCT_ID INT,
    TIME_STAMP TIMESTAMP,
    TRANSACTION_VALUE FLOAT,
    PRODUCT_ID INT,
    FOREIGN KEY (ACCT_ID) REFERENCES account(ACCT_ID),
    FOREIGN KEY (PRODUCT_ID) REFERENCES product(product_id)
)

Done.
Done.


[]

In [ ]:
%%sql
INSERT INTO education_fact 
SELECT education15.institution,year_D.year_id,location_D.location_id,admission_D.admission_id,retention_D.retention_id,sat_D.sat_id,debt_D.debt_id,fam_income_D.fam_income_id
FROM education15
JOIN
year_D
ON education15.year=year_D.year
JOIN
location_D
ON 
education15.city=location_D.city
AND education15.stabbr=location_D.state
AND education15.zip=location_D.zip
JOIN
admission_D
ON
education15.adm_rate=admission_D.adm_rate
AND education15.adm_rate_all=admission_D.adm_rate_all
JOIN
retention_D
ON
education15.ret_ft4=retention_D.ret_ft4
AND education15.ret_ftl4=retention_D.ret_ftl4_all
AND education15.ret_pt4=retention_D.ret_pt4
AND education15.ret_ptl4=retention_D.ret_ptl4
JOIN
sat_D
ON
education15.sat_avg=sat_d.sat_avg
AND education15.sat_avg_all=sat_d.sat_avg_all
JOIN
debt_d
ON
education15.debt_mdn=debt_D.debt_md
AND education15.grad_debt_mdn=debt_D.grad_debt_md
AND education15.wdraw_debt_mdn=debt_D.wdraw_debt_md
AND education15.lo_inc_debt_mdn=debt_D.lo_inc_debt_md
AND education15.md_inc_debt_mdn=debt_D.md_inc_debt_md
AND education15.hi_inc_debt_mdn=debt_D.hi_inc_debt_md
JOIN
fam_income_D
ON
education15.inc_pct_lo=fam_income_D.inc_pct_lo
AND education15.inc_pct_m1=fam_income_D.inc_pct_m1
AND education15.inc_pct_m2=fam_income_D.inc_pct_m2
AND education15.inc_pct_h1=fam_income_D.inc_pct_h1
AND education15.inc_pct_h2=fam_income_D.inc_pct_h2

